# 基本的图算法

In [33]:
import Queue

class Node:
    def __init__(self, value=None):
        self.value = value
        # color is "W" for white, "G" for gray, "B" for black
        self.color = "W"
        #depth means the length to root in breadth_first_search
        #and find time in depth_first_search
        self.depth = None
        self.final = None #final time used in depth_first_search
        self.pre = None

class Graphy:
    def __init__(self):
        self.node_neighbors = {} #map list
        self.visited = {}
        self.time = 0
    
    def add_nodes(self, nodelist):
        for node in nodelist:
            self.add_node(node)
    
    def add_node(self, node):
        if not node in self.nodes():
            self.node_neighbors[node] = []
    
    def add_edge(self, edge):
        u,v = edge
        if(v not in self.node_neighbors[u]) and ( u not in self.node_neighbors[v]):
            self.node_neighbors[u].append(v)
            
            if(u != v):
                self.node_neighbors[v].append(u)
                
    def nodes(self):
        return self.node_neighbors.keys()
    
    #广度优先遍历
    def breadth_first_search(self, s):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.pre = None
        s.color = "G"
        s.depth = 0
        s.pre = None
        Q = Queue.Queue()
        Q.put(s)
        while(not Q.empty()):
            u = Q.get()
            for v in self.node_neighbors[u]:
                if("W" == v.color):
                    v.color = "G"
                    v.depth = u.depth+1
                    v.pre = u
                    Q.put(v)
            u.color = "B"

    #深度优先遍历
    def depth_first_search(self, s):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.final = None
            node.pre = None
        self.time = 0
        for node in self.node_neighbors.keys():
            if("W" == node.color):
                self.dfs_visit(node)
    def dfs_visit(self, u):
        self.time += 1
        u.depth = self.time
        u.color = "G"
        for node in self.node_neighbors[u]:
            if("W" == node.color):
                node.pre = u
                self.dfs_visit(node)
        u.color = "B"
        self.time += 1
        u.final = self.time
        
    
g = Graphy()
nodes = [Node(i+1) for i in range(8)]
g.add_nodes(nodes)
g.add_edge((nodes[0], nodes[1]))
g.add_edge((nodes[0], nodes[2]))
g.add_edge((nodes[1], nodes[3]))
g.add_edge((nodes[1], nodes[4]))
g.add_edge((nodes[3], nodes[7]))
g.add_edge((nodes[4], nodes[7]))
g.add_edge((nodes[2], nodes[5]))
g.add_edge((nodes[2], nodes[6]))
g.add_edge((nodes[5], nodes[6]))

print("breadth first search:")
g.breadth_first_search(nodes[0])
for node in g.nodes():
    print (node.value, node.color, node.depth)

print("depth first search:")
g.depth_first_search(nodes[0])
for node in g.nodes():
    print (node.value, node.color, node.depth, node.final)
 

breadth first search:
(3, 'B', 1)
(5, 'B', 2)
(6, 'B', 2)
(1, 'B', 0)
(7, 'B', 2)
(8, 'B', 3)
(4, 'B', 2)
(2, 'B', 1)
depth first search:
(3, 'B', 1, 16)
(5, 'B', 6, 7)
(6, 'B', 12, 15)
(1, 'B', 2, 11)
(7, 'B', 13, 14)
(8, 'B', 5, 8)
(4, 'B', 4, 9)
(2, 'B', 3, 10)
